In [18]:
import os
import numpy as np # linear algebra
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import PIL
import PIL.Image
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [19]:
import numpy as np
import cv2

# path to the database - change it if needed
path = 'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\'

ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight',
          'noglasses', 'normal', 'rightlight','sad',
          'sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png' # file extension is png

# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)

# Chúng ta sẽ resize tất cả ảnh về kích thước 150x150
new_h, new_w = 150, 150

N = len(states) * 15 # Tổng số ảnh
print(N, new_h, new_w)

# Khởi tạo mảng để lưu trữ dữ liệu ảnh và nhãn sau khi resize
X = np.zeros((N, new_h, new_w, 1)) # Mảng này bây giờ có kích thước mới
Y = np.zeros(N) # Vector nhãn không đổi

# collect all data
cnt = 0

# lặp qua từng người và từng trạng thái cảm xúc
for person_id in range(1, 16):
    for state in states:
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix

        # open the file and read as grey image
        im = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)

        # resize and normalize the image
        resized_im = cv2.resize(im, (new_h, new_w)).astype(np.float32) / 255.0

        # thay vì reshape thành vector, ta chỉ cần mở rộng chiều
        X[cnt, :, :, 0] = resized_im
        Y[cnt] = states.index(state)

        cnt += 1

Y = Y.astype(int)

# Now X is in the format needed for a CNN: (num_images, height, width, num_channels)

165 150 150


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
import tensorflow as tf
from tensorflow import keras

def gen_model():
    # Define & compile the model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(new_h, new_w, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(rate=0.15),  # adding dropout regularization to deal with overfitting

        # The second convolution
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(rate=0.1),

        # The third convolution
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(rate=0.10),

        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),

        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),

        # Output layer with 11 neurons for the 11 classes
        tf.keras.layers.Dense(11, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


In [22]:
model = gen_model()
model.summary()

c:\Users\FPT SHOP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     9,470,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 11)             │         5,643 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,499,403 (36.24 MB)

 Trainable params: 9,499,403 (36.24 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(X_train, y_train, batch_size=2, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 13s 179ms/step - accuracy: 0.0534 - loss: 2.8198 - val_accuracy: 0.0303 - val_loss: 2.4020
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 11s 165ms/step - accuracy: 0.0965 - loss: 2.3988 - val_accuracy: 0.0303 - val_loss: 2.4047
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 13s 189ms/step - accuracy: 0.1312 - loss: 2.4019 - val_accuracy: 0.0909 - val_loss: 2.4062
Epoch 4/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 12s 183ms/step - accuracy: 0.1047 - loss: 2.4196 - val_accuracy: 0.0303 - val_loss: 2.4155
Epoch 5/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 12s 178ms/step - accuracy: 0.1345 - loss: 2.3926 - val_accuracy: 0.0000e+00 - val_loss: 2.4338
Epoch 6/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 11s 167ms/step - accuracy: 0.2089 - loss: 2.3042 - val_accuracy: 0.0606 - val_loss: 2.9447
Epoch 7/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 150ms/step - accuracy: 0.3432 - loss: 1.8761 - val_accuracy: 0.1212 - val_loss: 2.6121
Epoch 8/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 13s 193ms/step - accuracy: 0.4511 - loss: 1.5697 - val_